# Library Imports

In [24]:
import slideio
import shapely
import PIL
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from shapely import Polygon
from PIL import Image
import cv2
import json
import os
import openslide
import math
from tiatoolbox.tools import stainnorm
from PIL import Image
from tiatoolbox.tools.stainnorm import MacenkoNormalizer
from pathlib import Path


# Tubule Extraction from QuPath Annotations

## How to export data from QuPath

1. Open **File**
2. Open **Export objects as GeoJSON**.
3. Export as `All objects` or `Selected objects` (use this if you only want one class).
4. Enable `Pretty JSON` and `Export as FeatureCollection`.
5. Compression as `None`
6. Click **OK** to save the file.

## Loading QuPath GeoJSON into a DataFrame

Load all QuPath GeoJSON annotation files from a folder into one DataFrame

In [ ]:
# Folder where the geosjon files are saved
folder = r"Annotations json"  
dfs = []

for filename in os.listdir(folder):
    filepath = os.path.join(folder, filename)
    with open(filepath, "r", encoding="utf-8") as f:
        data = json.load(f)

        # Flatten features to DataFrame
        df = pd.json_normalize(data["features"])
        df["file"] = filename
        dfs.append(df)

# Merge to one dataframe
annot_df = pd.concat(dfs, ignore_index=True)

## Image file paths

Get the different image file names

In [26]:
image_names = annot_df['file'].unique()

Dictinoary with image-file-paths

In [27]:
# Folder with images
image_folder = r"images"

image_file_paths = dict()
for image in image_names:
    name = os.path.splitext(image)[0]  # remove .geojson
    svs_file = f"{name}.svs"
    image_path = os.path.join(image_folder, svs_file)
    image_file_paths[image] = image_path

## Exploring the data

The different classes 

In [28]:
# Finding classes
class_rows = annot_df['properties.classification.name'].tolist()
classes = np.array(class_rows)

In [29]:
# Count number in class
unique_classes, counts = np.unique(classes, return_counts=True)

for cls, count in zip(unique_classes, counts):
    print(f"{cls}: {count}")


Acute damage: 204
Atrophy: 184
Chronic damage: 158
Normal tubule: 263
ROI: 12
nan: 11


We focus on four target classes: Acute damage, Chronic damage, Normal tubule, and Atrophy, and remove all other classes.

In [30]:
wanted_classes = ['Acute damage', "Chronic damage", "Normal tubule", "Atrophy"]
mask = annot_df['properties.classification.name'].isin(wanted_classes)

wanted = mask.sum()
remove = (~mask).sum()
annot_df = annot_df[mask].reset_index(drop=True)

print(f'number of wanted rows: {wanted}')
print(f'number of rows removed: {remove}')


number of wanted rows: 809
number of rows removed: 23


Class distrubution

In [ ]:
class_rows = annot_df['properties.classification.name'].tolist()
classes = np.array(class_rows)

# Count number in class
unique_classes, counts = np.unique(classes, return_counts=True)
sum_classes = counts.sum()
for cls, count in zip(unique_classes, counts):
    prosent = count/sum_classes *100
    print(f"{cls}: {count} ({prosent:.3f}%)")


Acute damage: 204 (25.216%)
Atrophy: 184 (22.744%)
Chronic damage: 158 (19.530%)
Normal tubule: 263 (32.509%)


Check wether some annotations are multipolygon

In [32]:
value_counts = annot_df['geometry.type'].value_counts()
value_counts


geometry.type
Polygon         800
MultiPolygon      9
Name: count, dtype: int64

## Compute bounding boxes

Each individual patch gets a bounding box, defined as the smallest axis-aligned rectangle that fully contains the tubule.  
The coordinates of these boxes are computed in the format **(x, y, w, h)**, where:
- x, y = top-left corner
- w, h = width and height


### Bounding box

Bounding boxes are computed for each tubule, and the coordinates are appended to a list named `bbox_to_plot`.


In [33]:
bbox_list = []
bbox_to_plot = []
class_name = []
polygon_coord = []
file_names = []
polygon_type = []


for index, row in annot_df.iterrows():
    name = row['properties.classification.name']      # name
    class_name.append(name)
    image_name = row['file']                          # file-name
    file_names.append(image_name)
    geom_type = row['geometry.type']                  # geometry
    polygon_type.append(geom_type)
    coordinates = row['geometry.coordinates']        # coordinates
    polygon_coord.append(coordinates)


    if geom_type == 'MultiPolygon':
        coord = coordinates[0][0]  
    else:
        coord = coordinates[0]
    polygon = Polygon(coord)


    # Bounding box with margin
    bbox = list(polygon.bounds)  # (minx, miny, maxx, maxy)
    bbox_list.append(bbox)
    
    bbox = [math.ceil(x) for x in bbox]
    bbox[2] = bbox[2] - bbox[0]  # width
    bbox[3] = bbox[3] - bbox[1]  # height
    
    margin_width = max(1, math.ceil(bbox[2]*0.01))
    margin_height = max(1, math.ceil(bbox[3]*0.01))
    bbox[0], bbox[1] = bbox[0] - margin_width, bbox[1] - margin_height
    bbox[2], bbox[3] = bbox[2] + (2*margin_width), bbox[3] + (2*margin_height)
    bbox_to_plot.append(tuple(bbox))



### Centered bounding box

These bounding boxes are computed so that the tubule center is placed at the center of the box. The coordinates of each box are appended to a list called `centered_patches`.

In [34]:
centered_patches = []
for index, row in annot_df.iterrows():
    geom_type = row['geometry.type']
    coordinates = row['geometry.coordinates']
    
    if geom_type == 'MultiPolygon':
        coord = coordinates[0][0]
    else:
        coord = coordinates[0]
    
    polygon = Polygon(coord)
    cx, cy = polygon.centroid.x, polygon.centroid.y
    
    height = bbox_to_plot[index][3] 
    width = bbox_to_plot[index][2] 
    size = max(width, height) + 70   # The lagrest of width and height, + 70 pixel margin
    half_size = size // 2
    
    x0, y0 = int(cx - half_size), int(cy - half_size)
    centered_patches.append((x0, y0, size, size))
    

## Compact DataFrame

Visual_df is a DataFrame with only the information necessary for visualizing/plotting

In [35]:
# Visual_df is a DataFrame with only the information necessary for visualizing/plotting
visual_df = pd.DataFrame({
    'bounding_box': bbox_list,
    'bounding_box_for_plot': bbox_to_plot,
    'centered_patch': centered_patches,
    'class_name': class_name,
    'polygon_coord':polygon_coord, 
    'file': file_names,
    'polygon_type': polygon_type
})

visual_df

,bounding_box,bounding_box_for_plot,centered_patch,class_name,polygon_coord,file,polygon_type
0,"[50450.0, 17426.0, 50666.0, 17706.0]","(50447, 17423, 222, 286)","(50384, 17383, 356, 356)",Normal tubule,"[[[50507, 17426], [50503, 17428], [50500, 1742...",2016_220543_ANON.geojson,Polygon
1,"[50768.0, 17737.0, 51010.0, 17974.0]","(50765, 17734, 248, 243)","(50722, 17697, 318, 318)",Normal tubule,"[[[50860, 17737], [50858, 17738], [50855, 1773...",2016_220543_ANON.geojson,Polygon
2,"[50979.0, 17543.0, 51206.0, 17828.0]","(50976, 17540, 233, 291)","(50906, 17503, 361, 361)",Normal tubule,"[[[51081, 17543], [51079, 17544], [51077, 1754...",2016_220543_ANON.geojson,Polygon
3,"[50424.0, 17693.0, 50545.0, 17830.0]","(50422, 17691, 125, 141)","(50378, 17654, 211, 211)",Normal tubule,"[[[50476, 17693], [50474, 17694], [50470, 1769...",2016_220543_ANON.geojson,Polygon
4,"[50526.0, 17612.0, 50770.0, 18052.0]","(50523, 17607, 250, 450)","(50389, 17593, 520, 520)",Normal tubule,"[[[50729, 17612], [50727, 17613], [50725, 1761...",2016_220543_ANON.geojson,Polygon
...,...,...,...,...,...,...,...
804,"[54936.0, 25912.0, 55625.0, 26401.0]","(54929, 25907, 703, 499)","(54918, 25786, 773, 773)",Chronic damage,"[[[55545, 25912], [55544, 25913], [55540, 2591...",2021_222456_ANON.geojson,Polygon
805,"[55620.0, 26939.0, 55842.0, 27270.5]","(55617, 26935, 228, 340)","(55519, 26898, 410, 410)",Chronic damage,"[[[55704, 26939], [55703, 26940], [55701, 2694...",2021_222456_ANON.geojson,Polygon
806,"[55155.0, 26269.0, 55483.0, 26589.0]","(55151, 26265, 336, 328)","(55107, 26234, 406, 406)",Chronic damage,"[[[55470, 26269], [55469, 26270], [55464, 2627...",2021_222456_ANON.geojson,Polygon
807,"[55427.0, 26238.0, 55700.0, 26493.0]","(55424, 26235, 279, 261)","(55383, 26189, 349, 349)",Chronic damage,"[[[55554, 26238], [55552, 26239], [55550, 2624...",2021_222456_ANON.geojson,Polygon


## Cropping histology slides into smaller regions

Some tubules lie outside the current region of interest (ROI), so we can’t use it to crop the histological slide. We need a new ROI that includes all tubules. We’ll compute the global bounding box by taking the minimum and maximum x and y coordinates across all tubule annotations, then derive the width, height and top-left corner. Finally, we store the crop info in the dictionary `image_slides`.

In [36]:
image_slides = dict()
margin = 100

for i in range(len(image_names)):
    name = image_names[i]
    df = visual_df[visual_df['file'] == name]

    min_x_val = math.floor(min(bbox[0] for bbox in df['bounding_box']))-margin
    min_y_val = math.floor(min(bbox[1] for bbox in df['bounding_box']))-margin
    max_x_val = math.ceil(max(bbox[2] for bbox in df['bounding_box']))+margin
    max_y_val = math.ceil(max(bbox[3] for bbox in df['bounding_box']))+margin
    
    width = int(abs(max_x_val - min_x_val))
    height = int(abs(max_y_val - min_y_val))
    top_corner = (min_x_val, min_y_val)  # the top-left corner of the patch

    image_slides[name] = {
        "file": name,
        "min_x_val": min_x_val,
        "min_y_val": min_y_val,   
        "max_x_val": max_x_val,
        "max_y_val": max_y_val,
        "width": width,
        "height": height,
        "top_corner": top_corner
    }



The histology slides are cropped using the ROI parameters stored in image_slides. The cropped images are saved in the dictionary `slides_cropped`

In [37]:
slides_cropped = dict()
for name, dimensions in image_slides.items():
    top_corner = dimensions['top_corner']
    width = dimensions['width']
    height = dimensions['height']
    path = image_file_paths[name]
    slide = openslide.OpenSlide(path)
    slides_cropped[name] = slide.read_region(location=top_corner, level=0, size=(width, height))

## Create folders for patches

You may not use all outputs. Set True on the folders you need, **but be careful** – some folders depend on others.

In [49]:
# Folder where the folders with data get saved
OUT_DIR = Path("../data")

ENABLED = {
    "tubule_patches": True,
    "tubule_patches_scaled": False,   # depends on tubule_patches
    "center_patches": True,           # depends on tubule_patches
    "masked_patches_center": True,    # depends on tubule_patches + center_patches
    "masked_patches": True,           # depends on tubule_patches
    "masked_patches_scaled": False,   # depends on tubule_patches + masked_patches
}

folders = [key for key, val in ENABLED.items() if val]
for name in folders:
    (OUT_DIR / name).mkdir(parents=True, exist_ok=True)



## tubule_patches

In [50]:
name_to_prefix = {
    "Normal tubule": "normal_tubule",
    "Acute damage": "acute_damage",
    "Chronic damage": "chronic_damage",
    "Atrophy": "atrophy"
}


for i in range(len(visual_df)):
    file = visual_df.iloc[i, 5]
    slide = slides_cropped[file]
    name = visual_df.iloc[i,3]
    prefix = name_to_prefix.get(name) 
    
    # Convert global coordinates to local bbox coordinates
    global_x, global_y, width, height = visual_df.iloc[i, 1]  
    region_x, region_y = image_slides[file]['top_corner']
    local_x = global_x-region_x
    local_y = global_y-region_y
    
    tile = slide.crop((local_x,local_y,local_x+width,local_y+height))
    tile = tile.convert("RGB") 
    tile.save(f'../data/tubule_patches/{prefix}_p{i}.png', 'PNG') # save image based on class-name



## Color normalization of tubule_patches

Find a target image and use it as referance

In [51]:
# Folder with the target image
target_img = np.array(Image.open(r"target_image/target_image.png"))

normaliser = MacenkoNormalizer()
normaliser.fit(target_img)

Color noemalizing the tubule_patches

In [52]:
# Tubule patches
input_dir = r"../data/tubule_patches"
output_dir = r"../data/tubule_patches"
os.makedirs(output_dir, exist_ok=True)

# Iterate through all images in the input directory
for filename in os.listdir(input_dir):
    
    # Open the image 
    path = os.path.join(input_dir, filename)
    img = Image.open(path)   
    img_np = np.array(img)

    # Apply color normalization
    img_norm = normaliser.transform(img_np)
    img_norm_img = Image.fromarray(img_norm)

    # Save the normalized image to the output directory 
    img_norm_img.save(os.path.join(output_dir, filename))

## center_patches

In [53]:
input_dir = Path(r"../data/tubule_patches")
output_dir = Path(r"../data/center_patches")
os.makedirs(output_dir, exist_ok=True)
out_dir = Path("../data/center_patches")
center_patches = True

for filename in os.listdir(input_dir):
    
    if not out_dir.exists(): 
        center_patches = False
        continue
    
    path = input_dir / filename
    
    # Parse index from filename
    nr = int(path.stem.split("_")[-1][1:])

    # Convert global coordinates to local bbox coordinates
    bbox_x, bbox_y, bbox_w, bbox_h = visual_df.iloc[nr, 1]
    global_x, global_y, width, height = visual_df.iloc[nr, 2]
    local_x = int(global_x - bbox_x)
    local_y = int(global_y - bbox_y)
    
    # Crop and save
    with Image.open(path) as img:
        tile = img.crop((local_x, local_y, local_x + int(width), local_y + int(height)))
        tile.save(output_dir / filename)

## tubul_patches_scaled

In [54]:
# scaling images
target_size = (224, 224)  # width, height
input_folder = "../data/tubule_patches"

for filename in os.listdir(input_folder):
    img_path = os.path.join(input_folder, filename)
    img = cv2.imread(img_path)

    #change size
    img_resized = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA)
    name, ext = os.path.splitext(filename)
    new_filename = f"{name}_scaled{ext}"

    save_path = os.path.join("../data/tubule_patches_scaled", new_filename)
    cv2.imwrite(save_path, img_resized)


## masked_patches

In [55]:
file_dict = {
    "Normal tubule": "normal_tubule",
    "Acute damage": "acute_damage",
    "Chronic damage": "chronic_damage",
    "Atrophy": "atrophy"}
out_dir = Path("../data/masked_patches")
masked_patch = True

#bbox patches   
for i in range(len(visual_df)):
    
    if not out_dir.exists(): 
        masked_patch = False
        continue

    name = visual_df.iloc[i,3]
    polygon_type = visual_df.iloc[i, 6]

    prefix = file_dict[name]
    filename = f'../data/tubule_patches/{prefix}_p{i}.png'
    new_name = f'../data/masked_patches/{prefix}_p{i}.png'

    image = cv2.imread(filename)
    bbox = visual_df.iloc[i, 1]
    bbox = tuple([int(x) for x in bbox])


    if polygon_type == "MultiPolygon":
        coords = visual_df['polygon_coord'][i][0][0]
    else: 
        coords = visual_df['polygon_coord'][i][0]

    poly = Polygon(coords)
    poly_expanded = poly.buffer(5)
    
    polygon_points = np.array(poly_expanded.exterior.coords).astype('int32')  # some were floats, convert all to int 
    polygon_points [:, 0] = (polygon_points [:, 0]-bbox[0])
    polygon_points [:, 1] = (polygon_points [:, 1]-bbox[1])

    mask = np.zeros(image.shape[:2], dtype=np.uint8) 
    cv2.fillPoly(mask, [polygon_points], (255))

    foreground = cv2.bitwise_and(image, image, mask=mask)

    im = Image.fromarray(foreground)
    cv2.imwrite(new_name, foreground) 


## masked_patches_scaled

In [56]:
# scaling images
target_size = (224, 224)  # width, height
input_folder = "../data/masked_patches"
out_dir = Path("../data/masked_patches_scaled")

if not out_dir.exists() or not masked_patch:
    pass
else: 
    for filename in os.listdir(input_folder):
        img_path = os.path.join(input_folder, filename)
        img = cv2.imread(img_path)

        #change size
        img_resized = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA)
        name, ext = os.path.splitext(filename)
        new_filename = f"{name}_scaled{ext}"

        save_path = os.path.join("../data/masked_patches_scaled", new_filename)
        cv2.imwrite(save_path, img_resized)


## masked_patches_center

In [57]:
# Lagrest tubulu size
max_width = max(box[2] for box in bbox_to_plot)
max_height = max(box[3] for box in bbox_to_plot)
max_size = max(max_width, max_height) + 70  # lagrest size + 70 pixel margin
half_size = size // 2

out_dir = Path("../data/masked_patches_center")

if not out_dir.exists() or not center_patches:
    pass
for i in range(len(visual_df)):

    name = visual_df.iloc[i,3]
    polygon_type = visual_df.iloc[i, 6]

    prefix = file_dict[name]
    filename = f'../data/center_patches/{prefix}_p{i}.png'
    new_name = f'../data/masked_patches_center/{prefix}_p{i}.png'

    image = cv2.imread(filename)
    bbox = visual_df.iloc[i, 2]
    bbox = tuple([int(x) for x in bbox])

    if polygon_type == "MultiPolygon":
        coords = visual_df['polygon_coord'][i][0][0]
    else: 
        coords = visual_df['polygon_coord'][i][0]

    poly = Polygon(coords)
    poly_expanded = poly.buffer(5)
    
    polygon_points = np.array(poly_expanded.exterior.coords).astype('int32')  # some were floats, convert all to int 
    polygon_points [:, 0] = (polygon_points [:, 0]-bbox[0])
    polygon_points [:, 1] = (polygon_points [:, 1]-bbox[1])

    mask = np.zeros(image.shape[:2], dtype=np.uint8) 
    cv2.fillPoly(mask, [polygon_points], (255))

    # calculate the connected componante:  cv2.conectedComponentWithStats connected--> calculate the box
    foreground = cv2.bitwise_and(image, image, mask=mask)
    
    size = centered_patches[i][2]
    d_size = (max_size-size)
    pad_each = d_size // 2
    top = left = pad_each
    bottom = right = d_size - pad_each         # assigns the extra pixel when d_size is odd

    padded = cv2.copyMakeBorder(
        foreground, top, bottom, left, right,
        borderType=cv2.BORDER_CONSTANT, value=(0, 0, 0)
    )

    cv2.imwrite(new_name, padded) 